# ЛР5 Кривохата Марія ІМ-21

In [55]:
import random
import re
import pandas as pd
import spacy
from spacy.cli import download
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from textblob import TextBlob

Створити програму, яка:
1. Зчитує заданий набір даних, виконує попередню обробку (токенізацію,
видалення стоп-слів, лематизацію і т.д.) за допомогою бібліотеки spaCy.

 -1 – негативний коментар, 0 – нейтральний, 1 –
позитивний. Використати опорні вектори

In [3]:
data_df = pd.read_csv('reddit.csv')
data_df.head(10)

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
5,you should all sit down together and watch the...,-1
6,was teens when discovered zen meditation was ...,1
7,jesus was zen meets jew,0
8,there are two varieties christians dogmatic th...,-1
9,dont worry about trying explain yourself just ...,1


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [5]:
data_df = data_df.dropna(subset=['clean_comment'])
data_df = data_df[data_df['clean_comment'].str.strip() != '']

In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37028 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37028 non-null  object
 1   category       37028 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 867.8+ KB


In [7]:
download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [26]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha and len(token.text) > 2:
            tokens.append(token.lemma_)
    return ' '.join(tokens)

data_df['clean_text'] = data_df['clean_comment'].apply(preprocess)
data_df.head(10)

,clean_comment,category,clean_text
0,family mormon have never tried explain them t...,1,family mormon try explain stare puzzle time ti...
1,buddhism has very much lot compatible with chr...,1,buddhism lot compatible christianity especiall...
2,seriously don say thing first all they won get...,-1,seriously don thing win complex explain normal...
3,what you have learned yours and only yours wha...,0,learn want teach different focus goal wrapping...
4,for your own benefit you may want read living ...,1,benefit want read living buddha live christ th...
5,you should all sit down together and watch the...,-1,sit watch simpson episode lisa buddhist simpso...
6,was teens when discovered zen meditation was ...,1,teen discover zen meditation undiagnosed bpd h...
7,jesus was zen meets jew,0,jesus zen meet jew
8,there are two varieties christians dogmatic th...,-1,variety christians dogmatic dwell word conserv...
9,dont worry about trying explain yourself just ...,1,worry try explain meditate regularly try hard ...


In [27]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37028 entries, 0 to 37248
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37028 non-null  object
 1   category       37028 non-null  int64 
 2   clean_text     37028 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


2. Розбиває дані на навчальні на тестові. Виконує аналіз настроїв за
допомогою алгоритмів класифікації (опорних векторів). Виводить матрицю невідповідностей та точність
моделі.

In [33]:
# Розділення на навчальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(
    data_df['clean_text'], data_df['category'], test_size=0.2, random_state=47
)

In [34]:
# Векторизація тексту
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_df=0.9,
    min_df=5,
    max_features=10000
)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Масштабування
scaler = StandardScaler(with_mean=False)
X_train_vec = scaler.fit_transform(X_train_vec)
X_test_vec = scaler.transform(X_test_vec)

In [35]:
svm_model = LinearSVC(max_iter=10000, random_state=42)
svm_model.fit(X_train_vec, y_train)
y_pred_svm = svm_model.predict(X_test_vec)

C:\Users\mkriv\anaconda3\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [38]:
# Матриця невідповідностей та точність SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("Матриця невідповідностей:")
print(conf_matrix_svm)
print(f"Точність: {accuracy_svm:.2f}")

Матриця невідповідностей:
[[ 923  265  482]
 [ 355 1766  482]
 [ 519  445 2169]]
Точність: 0.66


3. Використовує один з готових лексиконів, наприклад Textblob, для
аналізу оцінки настроїв. Також розраховує матрицю невідповідностей, та
точність моделі.

In [53]:
# Аналіз настроїв за допомогою TextBlob
def textblob_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return 1
    elif polarity < -0.1:
        return -1
    else:
        return 0

# Оцінка всіх коментарів у тестовому наборі
y_pred_blob = X_test.apply(textblob_sentiment)

In [54]:
# Матриця невідповідностей та точність TextBlob
conf_matrix_blob = confusion_matrix(y_test, y_pred_blob)
accuracy_blob = accuracy_score(y_test, y_pred_blob)

print("Матриця невідповідностей:")
print(conf_matrix_blob)
print(f"Точність: {accuracy_blob:.2f}")

Матриця невідповідностей:
[[1022  574   74]
 [  53 2483   67]
 [ 108 1066 1959]]
Точність: 0.74


4. Обирає три випадкові записи та виводить результати оцінки їх настрою
за пунктами 2) і 3).

In [57]:
random_indices = random.sample(range(len(X_test)), 3)

for idx in random_indices:
    original_text = X_test.iloc[idx]
    true_label = y_test.iloc[idx]
    svm_prediction = y_pred_svm[idx]
    blob_prediction = y_pred_blob.iloc[idx]
    
    print(f"\nТекст: {data_df['clean_text'].iloc[X_test.index[idx]]}")
    print(f"Реальна оцінка: {true_label}")
    print(f"Прогноз SVM: {svm_prediction}")
    print(f"Прогноз TextBlob: {blob_prediction}")


Текст: stop citizen change irrespective party power
Реальна оцінка: 0
Прогноз SVM: 0
Прогноз TextBlob: 0

Текст: rujhano mean
Реальна оцінка: -1
Прогноз SVM: -1
Прогноз TextBlob: 1

Текст: hardik score india run delivery
Реальна оцінка: 0
Прогноз SVM: 0
Прогноз TextBlob: 0
